In [73]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

In [74]:
# 라이브러리 불러오기
import os
import json
import shutil
from PIL import Image
import re
import time
import pickle
import tempfile
import cv2
from ultralytics import YOLO

## 예측 코드

In [86]:
# 1. 학습된 best.pt 모델 로드
model = YOLO('runs/detect/train40/weights/best.pt')

# 2. 예측하고자 하는 이미지 파일 경로
# image_path = '[BLUE]00127A_101213_004.jpg'
# image_path = '[BLUE]00219A_155106_002.jpg'
image_path = 'platecar3_downsample.jpg'
# image_path = '[BLUE]00309A_082918_012.jpg'

# 3. 이미지 불러오기
img = cv2.imread(image_path)

# 3. 이미지 다운샘플링 
downsampled_img = cv2.resize(img, (640, 480))

# # 5. 리사이즈된 이미지 임시 저장 (YOLO는 파일 경로를 입력받으므로, 리사이즈된 이미지를 임시 파일로 저장)
file_name, file_ext = os.path.splitext(image_path)
downsampled_image_path  = f"{file_name}_downsample{file_ext}"
cv2.imwrite(downsampled_image_path , downsampled_img)

# # 4. 이미지 크기를 1920x1080으로 리사이즈
# resized_img = cv2.resize(img, (1920, 1080))

# # 5. 리사이즈된 이미지 임시 저장 (YOLO는 파일 경로를 입력받으므로, 리사이즈된 이미지를 임시 파일로 저장)
# file_name, file_ext = os.path.splitext(image_path)
# temp_image_path = f"{file_name}_resized{file_ext}"
# cv2.imwrite(temp_image_path, resized_img)

# 이미지 예측
results = model.predict(source=downsampled_image_path , save=True, imgsz=640)  # your_image.jpg 경로에 맞게 수정

# 예측 결과가 저장된 폴더 경로 가져오기
save_dir = str(results[0].save_dir)

# 원본 이미지 불러오기
orig_img = results[0].orig_img

# 탐지된 차량 이미지를 저장된 결과 폴더에 저장
for i, vehicle_box in enumerate(results[0].boxes):
    cls = int(vehicle_box.cls)
    class_name = results[0].names[cls]
    

    # 차량 클래스일 경우만 저장
    if class_name in ['vehicle_car', 'vehicle_bus', 'vehicle_truck', 'vehicle_bike']:
        # 차량의 경계 상자 좌표 가져오기
        x1, y1, x2, y2 = map(int, vehicle_box.xyxy[0])
        
        # 경계 상자로 차량 이미지 크롭
        cropped_vehicle = orig_img[y1:y2, x1:x2]
        
        # 저장 경로 설정
        save_path = f"{save_dir}/vehicle_{i}.jpg"
        
        # 크롭한 차량 이미지를 저장
        cv2.imwrite(save_path, cropped_vehicle)
        print(f"탐지된 차량이 {save_path}에 저장되었습니다.")

# 결과 확인
for result in results:
    result.plot()  # 예측 결과 이미지 출력


image 1/1 /home/j-j11a303/platecar3_downsample_downsample.jpg: 480x640 (no detections), 8.0ms
Speed: 1.5ms preprocess, 8.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict80


In [87]:
# 4. 차량과 차선 객체 분류
vehicle_boxes = []
lane_boxes = []

for box in results[0].boxes:  # results[0]을 사용하여 첫 번째 결과에 접근
    cls = int(box.cls)  # 클래스 인덱스 가져오기
    class_name = results[0].names[cls]  # 클래스 이름 가져오기
    
    # 클래스 이름에 따라 차량과 차선을 구분
    if class_name in ['vehicle_car', 'vehicle_bus', 'vehicle_truck', 'vehicle_bike']:
        vehicle_boxes.append(box)
    elif class_name in ['lane_white', 'lane_blue', 'lane_yellow', 'lane_shoulder']:
        lane_boxes.append(box)

# 5. 차량과 차선 간의 관계 분석
def is_overlapping(box1, box2):
    # 각 경계 상자의 좌표 (x1, y1, x2, y2)
    x1_1, y1_1, x2_1, y2_1 = box1.xyxy[0].tolist()  # 차량 경계 상자 좌표
    x1_2, y1_2, x2_2, y2_2 = box2.xyxy[0].tolist()  # 차선 경계 상자 좌표
    
    # 겹침 여부 확인 (두 상자가 겹치는 경우 True 반환)
    return not (x2_1 < x1_2 or x2_2 < x1_1 or y2_1 < y1_2 or y2_2 < y1_1)


print(vehicle_boxes)

# 6. 불법 주차 여부 판단
for vehicle_box in vehicle_boxes:
    vehicle_coords = vehicle_box.xyxy  # 차량의 경계 상자 좌표
    
    for lane_box in lane_boxes:
        lane_coords = lane_box.xyxy  # 차선의 경계 상자 좌표
        
        # 차량이 차선 위에 있는지 확인 (경계 상자 간의 겹침 여부 계산)
        if is_overlapping(vehicle_box, lane_box):
            lane_type = results[0].names[int(lane_box.cls)]  # 차선의 종류
            print(f"차량이 {lane_type} 차선 위에 있습니다.")
            
            # 파란색 차선 위에 있으면 불법 주차로 판단
            if lane_type in  ['lane_blue', 'lane_yellow', 'lane_shoulder']:
                print("불법 주차입니다.")
            else:
                print("정상 주차입니다.")

[]


In [77]:
print(results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'vehicle_car', 1: 'vehicle_bus', 2: 'vehicle_truck', 3: 'vehicle_bike', 4: 'lane_white', 5: 'lane_blue', 6: 'lane_yellow', 7: 'lane_shoulder'}
obb: None
orig_img: array([[[131, 137, 136],
        [134, 139, 137],
        [171, 172, 168],
        ...,
        [187, 188, 184],
        [189, 190, 186],
        [192, 193, 189]],

       [[117, 123, 122],
        [152, 157, 155],
        [178, 180, 174],
        ...,
        [192, 193, 189],
        [191, 192, 188],
        [190, 191, 187]],

       [[156, 165, 162],
        [195, 202, 197],
        [193, 197, 191],
        ...,
        [190, 191, 187],
        [188, 189, 185],
        [186, 187, 183]],

       ...,

       [[150, 151, 147],
        [149, 150, 146],
        [126, 127, 123],
        ...,
        [169, 170, 166],
        [148, 149, 145],
        [149, 150, 146]],

       [[168, 170

In [31]:
print(results[0].boxes.cls[1])

tensor(1., device='cuda:0')
